In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qfrtutn1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-qfrtutn1
  Resolved https://github.com/huggingface/transformers.git to commit 3ec8171bedff6139a23dff192b9e8af33c1fca9a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6702991 sha256=f18763331f88850005df17b705ecbb9497ccb1b579b51c6f99fc91b43e937cf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_45azmd/wheels/f7/92/8c/752ff3bfcd3439805d8bbf641614da38ef3226e127ebea86ee
Successfully built transformers


In [ ]:
!unzip -u "/content/drive/My Drive/102_flower.zip" -d "/content/102_flower"

Streaming output truncated to the last 5000 lines.
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06553.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06554.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06555.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06556.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06557.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06559.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06560.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06561.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06563.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06564.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06565.jpg  
  inflating: /content/102_flower/102_flower/dataset/train/50/image_06566.jpg  
 

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

In [ ]:
import glob
list_of_files = glob.glob('dataset/train/*/*')

In [ ]:
f = open('cat_to_name.json')
labels = json.load(f)
labels

{'21': 'fire lily',
 '3': 'canterbury bells',
 '45': 'bolero deep blue',
 '1': 'pink primrose',
 '34': 'mexican aster',
 '27': 'prince of wales feathers',
 '7': 'moon orchid',
 '16': 'globe-flower',
 '25': 'grape hyacinth',
 '26': 'corn poppy',
 '79': 'toad lily',
 '39': 'siam tulip',
 '24': 'red ginger',
 '67': 'spring crocus',
 '35': 'alpine sea holly',
 '32': 'garden phlox',
 '10': 'globe thistle',
 '6': 'tiger lily',
 '93': 'ball moss',
 '33': 'love in the mist',
 '9': 'monkshood',
 '102': 'blackberry lily',
 '14': 'spear thistle',
 '19': 'balloon flower',
 '100': 'blanket flower',
 '13': 'king protea',
 '49': 'oxeye daisy',
 '15': 'yellow iris',
 '61': 'cautleya spicata',
 '31': 'carnation',
 '64': 'silverbush',
 '68': 'bearded iris',
 '63': 'black-eyed susan',
 '69': 'windflower',
 '62': 'japanese anemone',
 '20': 'giant white arum lily',
 '38': 'great masterwort',
 '4': 'sweet pea',
 '86': 'tree mallow',
 '101': 'trumpet creeper',
 '42': 'daffodil',
 '22': 'pincushion flower',
 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
model.to(device)

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0): BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BlipEncoderLayer(
          (self_attn): BlipAttention(
 

In [ ]:
df = pd.DataFrame({"label": [], "file_name":[], "text": []})

In [ ]:
for i in tqdm(list_of_files):

  path_to_image = i
  raw_image = Image.open(path_to_image).convert('RGB')

  # unconditional image captioning
  inputs = processor(raw_image, return_tensors="pt").to(device)

  out = model.generate(**inputs)
  caption = processor.decode(out[0], skip_special_tokens=True)
  path_split = i.split("/")
  label = path_split[2]
  file_name = path_split[3]
  if (labels[label] not in caption) and ("flower" in caption):
    caption = caption.replace("flower", labels[label], 1)
  row = [label, file_name, caption]
  df.loc[len(df)] = row
  df.to_csv("metadata.csv", index=False)
  df.to_csv("drive/MyDrive/metadata.csv", index=False)


  0%|          | 0/6552 [00:00<?, ?it/s]